# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [15]:
# Use this cell to write your code for Task 1

import pandas as pd
df = pd.read_csv('house_sales.csv')
missing_city = df['city'].value_counts()['--']
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [16]:
# Use this cell to write your code for Task 2

df['area'] = df['area'].str.replace(' sq.m.', '').astype(float)
df['city'].replace('--', 'Unknown', inplace=True)
df.dropna(subset=['sale_price'], inplace=True)
df['sale_date'].fillna('2023-01-01', inplace=True)
df['months_listed'].fillna(df['months_listed'].mean().round(1), inplace=True)
df['bedrooms'].fillna(df['bedrooms'].mean().round(), inplace=True)
df['house_type'].replace({'Det.': 'Detached', 'Semi': 'Semi-detached', 'Terr.': 'Terraced'}, inplace=True)
df['house_type'].fillna(df['house_type'].mode()[0], inplace=True)
df['area'].fillna(df['area'].mean().round(1), inplace=True)
df['sale_price'] = df['sale_price'].astype(int)
df['bedrooms'] = df['bedrooms'].astype(int)
clean_data = df
clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [17]:
# Use this cell to write your code for Task 3

price_by_rooms = df.groupby('bedrooms').agg(
    avg_price=('sale_price', 'mean'),
    var_price=('sale_price', 'var')
).reset_index()
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [18]:
# Use this cell to write your code for Task 4

from sklearn.linear_model import LinearRegression
train = pd.read_csv('train.csv')
validation = pd.read_csv('validation.csv')


city_mapping = {'Teasdale': 0, 'Silvertown': 1, 'Poppleton': 2, 'Riverford': 3}
house_type_mapping = {'Detached': 0, 'Semi-detached': 1, 'Terraced': 2}


train['city'] = train['city'].map(city_mapping)
validation['city'] = validation['city'].map(city_mapping)

train['house_type'] = train['house_type'].map(house_type_mapping)
validation['house_type'] = validation['house_type'].map(house_type_mapping)


train['sale_date'] = pd.to_datetime(train['sale_date'])


train['day'] = train['sale_date'].dt.day
train['month'] = train['sale_date'].dt.month
train['year'] = train['sale_date'].dt.year

validation['sale_date'] = pd.to_datetime(validation['sale_date'])


validation['day'] = validation['sale_date'].dt.day
validation['month'] = validation['sale_date'].dt.month
validation['year'] = validation['sale_date'].dt.year

X_train = train.drop(columns=['house_id', 'sale_price', 'sale_date'])
y_train = train['sale_price']
X_validation = validation.drop(columns=['house_id', 'sale_date'])


baseline_model = LinearRegression()
baseline_model.fit(X_train, y_train)


validation['price'] = baseline_model.predict(X_validation)


base_result = validation[['house_id', 'price']]
base_result['price'] = base_result['price'].round(1)


print(base_result.head())

   house_id     price
0   1331375  122895.6
1   1630115  304900.6
2   1645745  383484.2
3   1336775  122490.5
4   1888274  270700.1


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [19]:
# Use this cell to write your code for Task 5
from sklearn.ensemble import RandomForestRegressor


comparison_model = RandomForestRegressor()
comparison_model.fit(X_train, y_train)


validation['price'] = comparison_model.predict(X_validation)


compare_result = validation[['house_id', 'price']]
compare_result['price'] = compare_result['price'].round(1)


print(compare_result.head())

   house_id     price
0   1331375   81688.2
1   1630115  308495.8
2   1645745  402136.0
3   1336775  107169.3
4   1888274  266104.5
